In [196]:
import pyodbc
import re
import json

import numpy as np
import pandas as pd
import tensorflow as tf

from sqlalchemy import create_engine
from sklearn.metrics.pairwise import cosine_similarity

from collections import defaultdict
from gensim import corpora, models, similarities
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex
from gensim.models.word2vec import Word2Vec

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from googletrans import Translator

from FlaskApp.transform import *

from job_model import JobModel
from app_model import AppModel

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [197]:
translator = Translator(service_urls=['translate.googleapis.com'])
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [198]:
def remove_stopwords(stopwords, texts):
    split_text = texts.split()
    clean_text = []
    for text in split_text:
        if text not in stopwords:
            clean_text.append(text)
    return ' '.join(clean_text)

In [199]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

In [200]:
user = 'huda'
password = 'Vancha12'
host = '127.0.0.1'
port = 1433
database = 'HRSystemDB'


def get_connection():         
    return create_engine(
        url=f"mssql+pyodbc://{user}:{password}@{host}:{port}/{database}?driver=SQL Server",
    )

engine = get_connection()
conn = engine.connect()

In [201]:
df_job = pd.DataFrame(engine.execute(
    """
    SELECT Job.JobID, Job.UsiaMax, Job.SalaryMin, Job.SalaryMax, City.Name AS CityName, Province.Name AS ProvinceName, EducationLevel.EducationLevelName, Major.MajorName, Job.DriverLicenseType, Job.UsingGlasses, Job.Gender, Job.MaritalStatus, Job.JobTitle, FunctionPosition.FunctionPositionName, Job.Description, Job.Requirement
    FROM (((((Job
    RIGHT JOIN FunctionPosition ON Job.FunctionPositionID = FunctionPosition.FunctionPositionID)
    RIGHT JOIN EducationLevel ON Job.EducationLevelID = EducationLevel.EducationLevelID)
    RIGHT JOIN City ON Job.CityID = City.CityID)
    RIGHT JOIN Province ON Job.ProvinceID = Province.ProvinceID)
    RIGHT JOIN Major ON Job.MajorID = Major.MajorID)
    """
))

df_function = pd.DataFrame(engine.execute(
    """
    SELECT FunctionPositionID, FunctionPositionName
    FROM FunctionPosition
    """
))

df_education = pd.DataFrame(engine.execute(
    """
    SELECT EducationLevelID, EducationLevelName
    FROM EducationLevel
    """
))

df_city = pd.DataFrame(engine.execute(
    """
    SELECT CityID, Name AS CityName
    FROM City
    """
))

df_province = pd.DataFrame(engine.execute(
    """
    SELECT ProvinceID, Name AS ProvinceName
    FROM Province
    """
))

df_major = pd.DataFrame(engine.execute(
    """
    SELECT MajorID, MajorName
    FROM Major
    """
))

df_applicant = pd.DataFrame(engine.execute(
    f"""
    SELECT Applicant.ApplicantID, Applicant.Dob, Applicant.ExpectedSalary, City.Name AS CityName, Province.Name AS ProvinceName, Applicant.DriverLicenseType, Applicant.IsUsingGlasses, Applicant.Gender, Applicant.MaritalStatus, Applicant.Strengthness
    FROM (((Applicant
    RIGHT JOIN City ON Applicant.CurrentAddressCityID = City.CityID)
    RIGHT JOIN Province ON Applicant.CurrentAddressProvinceID = Province.ProvinceID)
    LEFT JOIN Pipeline ON Applicant.ApplicantID = Pipeline.ApplicantID)
    """
))

df_applicant_education = pd.DataFrame(engine.execute(
    f"""
    SELECT ApplicantEducation.ApplicantID, ApplicantEducation.DateStart, ApplicantEducation.DateEnd, EducationLevel.EducationLevelName, Major.MajorName
    FROM (((ApplicantEducation
    RIGHT JOIN EducationLevel ON ApplicantEducation.EducationLevelID = EducationLevel.EducationLevelID)
    RIGHT JOIN Major ON ApplicantEducation.MajorID = Major.MajorID)
    LEFT JOIN Pipeline ON ApplicantEducation.ApplicantID = Pipeline.ApplicantID)
    """
))

df_applicant_experience = pd.DataFrame(engine.execute(
    f"""
    SELECT ApplicantExperience.ApplicantID, ApplicantExperience.DateFrom, ApplicantExperience.DateTo, ApplicantExperience.Position, ApplicantExperience.JobDescription
    FROM (ApplicantExperience
    LEFT JOIN Pipeline ON ApplicantExperience.ApplicantID = Pipeline.ApplicantID)
    """
))

In [202]:
'''save json'''
# with open('data/englishwords.json', 'w') as file:
#     json.dump(englishwords, file)

# with open('data/slangwords.json', 'w') as file:
#     json.dump(slangwords, file)

'''load json'''
with open('data/englishwords.json', 'r') as file:
    englishwords = json.load(file)

with open('data/slangwords.json', 'r') as file:
    slangwords = json.load(file)

'''load csv'''
stopwords = [i[0] for i in pd.read_csv('data/stopwords.csv').astype(str).values] + ['dst']

In [203]:
# # pendekatan baru: tokenizing kemudian ditranslate satu-satu dan di cek satu-satu terhadap kata yang gagal di translate kemudian membuat set dari hasil translate token tesebut
# description_corpus = []
# for token in df_job.Description.values:
#     description_corpus.append(token)
# else:
    
#     description_corpus = set(description_corpus)

In [204]:
df_applicant.head(1)

,ApplicantID,Dob,ExpectedSalary,CityName,ProvinceName,DriverLicenseType,IsUsingGlasses,Gender,MaritalStatus,Strengthness
0,2.0,2018-04-10 00:00:00.0000000,0.0,SURABAYA,JAWA TIMUR,A,False,Male,None,strength


In [205]:
'''applicant'''
df_applicant = df_applicant.dropna(subset=['ApplicantID'])
df_applicant.ApplicantID = df_applicant.ApplicantID.astype(int)
df_applicant = df_applicant.drop_duplicates()
df_applicant = df_applicant.fillna('')
# age column
df_applicant['Age'] = pd.to_datetime(
    df_applicant.Dob.map(pick_date).apply(lambda x: filter_date(x, 1958, 2006))
).map(get_age)

df_applicant.drop(columns=['Dob'], inplace=True)

df_applicant.Age = df_applicant.Age.fillna(0).astype(int)

In [206]:
df_applicant_education.head(1)

,ApplicantID,DateStart,DateEnd,EducationLevelName,MajorName
0,3,2011-07-01 00:00:00.0000000,2016-01-01 00:00:00.0000000,S1,SEMUA JURUSAN


In [207]:
'''education'''
df_applicant_education = df_applicant_education.fillna('')
# datetime column
df_applicant_education.DateStart = pd.to_datetime(
    df_applicant_education.DateStart.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)
df_applicant_education.DateEnd = pd.to_datetime(
    df_applicant_education.DateEnd.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)
df_applicant_education = df_applicant_education[~(df_applicant_education.DateStart.isna()) & ~(df_applicant_education.DateEnd.isna())]
df_applicant_education = df_applicant_education.sort_values('DateStart').groupby(['ApplicantID']).agg('last')
df_applicant_education.drop(columns=['DateStart', 'DateEnd'], inplace=True)

In [208]:
df_applicant_experience.head(1)

,ApplicantID,DateFrom,DateTo,Position,JobDescription
0,3,2016-04-01 00:00:00.0000000,2017-04-01 00:00:00.0000000,Sales Executive,<p>- Maintenance Dealer (cek stock dealer apak...


In [209]:
'''experience'''
df_applicant_experience = df_applicant_experience.fillna('')

# datetime column
df_applicant_experience.DateFrom = pd.to_datetime(
    df_applicant_experience.DateFrom.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)
df_applicant_experience.DateTo = pd.to_datetime(
    df_applicant_experience.DateTo.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)
df_applicant_experience = df_applicant_experience[~(df_applicant_experience.DateFrom.isna()) & ~(df_applicant_experience.DateTo.isna())]

# add YearsOfExperience column
df_applicant_experience['YearsOfExperience'] = substract_months(
    df_applicant_experience.DateFrom, df_applicant_experience.DateTo
)
df_applicant_experience = df_applicant_experience.sort_values('DateFrom').groupby(['ApplicantID']).agg({
    'DateFrom': 'last',
    'DateTo': 'last',
    'JobDescription': ' '.join,
    'Position': ' '.join,
    'YearsOfExperience': 'sum',
})
df_applicant_experience.drop(columns=['DateFrom', 'DateTo'], inplace=True)

In [210]:
df_applicant.head(1)

,ApplicantID,ExpectedSalary,CityName,ProvinceName,DriverLicenseType,IsUsingGlasses,Gender,MaritalStatus,Strengthness,Age
0,2,0.0,SURABAYA,JAWA TIMUR,A,False,Male,,strength,0


In [211]:
'''merge'''
df_applicant = pd.merge(df_applicant, df_applicant_experience, on=['ApplicantID'])
df_applicant = pd.merge(df_applicant, df_applicant_education, on=['ApplicantID'])

In [212]:
'''PRE-PROCESSING APPLICANT'''
df_applicant.set_index(['ApplicantID'], inplace=True)

app_str = ['CityName', 'ProvinceName', 'EducationLevelName', 'MajorName', 'DriverLicenseType', 'Gender', 'MaritalStatus', 'Position', 'JobDescription', 'Strengthness']
app_num = ['Age', 'ExpectedSalary', 'YearsOfExperience']
app_bol = ['IsUsingGlasses']

df_applicant = df_applicant[app_num + app_bol + app_str]

'''str'''
df_applicant[app_str] = df_applicant[app_str].applymap(str.lower)

df_applicant.JobDescription = df_applicant.JobDescription.map(clean_text).map(maintain_alphabet).map(remove_single).map(remove_morespace)

df_applicant.Strengthness = df_applicant.Strengthness.map(clean_text).map(maintain_alphabet).map(remove_single).map(remove_morespace)

df_applicant.Position = df_applicant.Position
df_applicant.MajorName = df_applicant.MajorName

# concat
df_applicant.JobDescription = df_applicant.JobDescription.str.cat(
    df_applicant.Strengthness, sep=' '
)
df_applicant.rename(columns={'JobDescription': 'DescriptionStrengthness'}, inplace=True)
df_applicant.drop(columns=['Strengthness'], inplace=True)

'''bool'''
df_applicant.IsUsingGlasses = df_applicant.IsUsingGlasses.astype(str).map(str.lower)

c:\Users\eats\miniconda3\envs\p1\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [215]:
df_job.isna().sum()

UsiaMax                 0
SalaryMin               0
SalaryMax               0
CityName                0
ProvinceName            0
EducationLevelName      0
MajorName               0
DriverLicenseType       0
UsingGlasses            0
Gender                  0
MaritalStatus           0
JobTitle                0
FunctionPositionName    0
Description             0
Requirement             0
dtype: int64

In [214]:
job_str = ['CityName', 'ProvinceName', 'EducationLevelName', 'MajorName', 'DriverLicenseType', 'Gender', 'MaritalStatus', 'JobTitle', 'FunctionPositionName', 'Description', 'Requirement']
job_num = ['UsiaMax', 'SalaryMin', 'SalaryMax']
job_bol = ['UsingGlasses']

'''general'''
df_job.set_index(['JobID'], inplace=True)
df_job.fillna('', inplace=True)

'''str'''
df_job[job_str] = df_job[job_str].applymap(str.lower)
df_job.replace('none', '', inplace=True)

In [216]:
df_job.head(1)

,UsiaMax,SalaryMin,SalaryMax,CityName,ProvinceName,EducationLevelName,MajorName,DriverLicenseType,UsingGlasses,Gender,MaritalStatus,JobTitle,FunctionPositionName,Description,Requirement
JobID,,,,,,,,,,,,,,,
1,0.0,3300000.0,3600000.0,surabaya,jawa timur,d3,perhotelan,,False,,,night audit,night audit,under asst. front office manager at gunawangsa...,"<p>\r\n\r\n</p><div style=""language:en-us;marg..."


In [217]:
"""PRE-PROCESSING JOB"""
df_job.Description = df_job.Description.map(clean_text).map(maintain_alphabet).map(remove_single).map(remove_morespace)
df_job.Requirement = df_job.Requirement.map(clean_text).map(maintain_alphabet).map(remove_single).map(remove_morespace)

# concatenate JobTitle and FunctionPositionName to be textual feature together
df_job.JobTitle = df_job.JobTitle.str.cat(
    df_job.FunctionPositionName, sep=' '
)
df_job.rename(columns={'JobTitle': 'JobTitlePosition'}, inplace=True)
df_job.drop(columns=['FunctionPositionName'], inplace=True)

# concatenate Description and Requirement to be textual feature together
df_job.Description = df_job.Description.str.cat(
    df_job.Requirement, sep=' '
)
df_job.rename(columns={'Description': 'DescriptionRequirement'}, inplace=True)
df_job.drop(columns=['Requirement'], inplace=True)

df_job = df_job[~(df_job.DescriptionRequirement == ' ')]

'''int'''
df_job[job_num] = df_job[job_num].replace('', 0)
df_job[job_num] = df_job[job_num].astype(int)

# get mean from SalaryMin and SalaryMax
df_job.SalaryMin = (df_job.SalaryMax + df_job.SalaryMin) // 2
df_job.rename(columns={'SalaryMin': 'SalaryMean'}, inplace=True)
df_job.SalaryMean = df_job.SalaryMean.apply(lambda x: 0 if x < 1_000_000 else x)
df_job.drop(columns=['SalaryMax'], inplace=True)

'''bool'''
df_job.UsingGlasses = df_job.UsingGlasses.astype(str).map(str.lower)


# load table for vocabulary
df_function.FunctionPositionName = df_function.FunctionPositionName.map(str.lower).apply(lambda x: function_replacement(x)).map(remove_insideparentheses).map(remove_morespace).map(str.strip)
df_education.EducationLevelName = df_education.EducationLevelName.map(str.lower)
df_city.CityName = df_city.CityName.map(str.lower)
df_province.ProvinceName = df_province.ProvinceName.map(str.lower)
df_major.MajorName = df_major.MajorName.map(str.lower)

In [219]:
df_job.head(1)

,UsiaMax,SalaryMean,CityName,ProvinceName,EducationLevelName,MajorName,DriverLicenseType,UsingGlasses,Gender,MaritalStatus,JobTitlePosition,DescriptionRequirement
JobID,,,,,,,,,,,,
1,0,3450000,surabaya,jawa timur,d3,perhotelan,,false,,,night audit night audit,under asst front office manager at gunawangsa ...


In [220]:
df_job['Texts'] = df_job.JobTitlePosition + ' ' + df_job.DescriptionRequirement

In [221]:
df_applicant['Texts'] = df_applicant.Position + ' ' + df_applicant.DescriptionStrengthness

In [222]:
df_job.head(1)

,UsiaMax,SalaryMean,CityName,ProvinceName,EducationLevelName,MajorName,DriverLicenseType,UsingGlasses,Gender,MaritalStatus,JobTitlePosition,DescriptionRequirement,Texts
JobID,,,,,,,,,,,,,
1,0,3450000,surabaya,jawa timur,d3,perhotelan,,false,,,night audit night audit,under asst front office manager at gunawangsa ...,night audit night audit under asst front offic...


In [226]:
df_applicant.head(1)

,Age,ExpectedSalary,YearsOfExperience,IsUsingGlasses,CityName,ProvinceName,EducationLevelName,MajorName,DriverLicenseType,Gender,MaritalStatus,Position,DescriptionStrengthness,Texts
ApplicantID,,,,,,,,,,,,,,
3,29,4000000.0,1.833333,false,surabaya,jawa timur,s1,semua jurusan,c,male,single,sales executive whole sales credit marketing o...,maintenance dealer cek stock dealer apakah mas...,sales executive whole sales credit marketing o...


In [223]:
import gensim
import gensim.downloader as api
from gensim.models import doc2vec

# get dataset
# dataset = api.load("text8")
dataset = list(df_job.Texts.apply(lambda x: x.split()).values)
data =[]
for w in dataset:
    data.append(w)

# To train the model we need a list of tagged documents
def tagged_document(list_of_ListOfWords):
    for x, ListOfWords in enumerate(list_of_ListOfWords):
        yield doc2vec.TaggedDocument(ListOfWords, [x])

# training data
data_train = list(tagged_document(data))

In [224]:
# Initialize the model
d2v_model = doc2vec.Doc2Vec(vector_size=120, min_count=10, epochs=90)

# build the vocabulary
d2v_model.build_vocab(data_train)

# Train Doc2Vec model
d2v_model.train(data_train, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

2023-04-17 10:38:48,628 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d120,n5,w5,mc10,s0.001,t3>', 'datetime': '2023-04-17T10:38:48.628419', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-04-17 10:38:48,628 : INFO : collecting all words and their counts
2023-04-17 10:38:48,629 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-04-17 10:38:48,640 : INFO : collected 3703 word types and 1006 unique tags from a corpus of 1006 examples and 102767 words
2023-04-17 10:38:48,640 : INFO : Creating a fresh vocabulary
2023-04-17 10:38:48,647 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=10 retains 1450 unique words (39.16% of original 3703, drops 2253)', 'datetime': '2023-04-17T10:38:48.647114', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:

In [230]:
results = {}

for i in df_job.index:
    user_i = {}
    for j in df_applicant.index:
        similarity = d2v_model.similarity_unseen_docs(
            df_job.Texts.loc[i].split(),
            df_applicant.Texts.loc[j].split()
        )
        user_i[i] = similarity
    results[j] = user_i

KeyboardInterrupt: 

In [ ]:
results

In [ ]:
d2v_model.similarity_unseen_docs(df_job.Texts.index[27].split(), df_applicant.Texts.loc[15].split())

In [ ]:
# d2v_model.save('data/d2c_model.model')
# loaded_model = Word2Vec.load('data/d2c_model.model')

In [ ]:
d2v_model.infer_vector()

In [ ]:
d2v_model.similarity_unseen_docs()